In [1]:
from django_for_jupyter import init_django

init_django()

In [2]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from tabulate import tabulate
import advertools as adv
# import crawl

In [3]:
from precios.pi_supermercado import (
    get_url_robots,
    get_content_from_column,
    get_sitemap_from_url,
)

In [4]:
import time
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromiumService
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.utils import ChromeType
from selenium.webdriver.common.action_chains import ActionChains
import chromedriver_autoinstaller
from urllib.parse import urlparse

try:
    chromedriver_autoinstaller.install() 
except:
    print('Sin conexión')


from precios.models import Site, Campos, CamposEnSitio, SiteURLResults, Pages

from precios.pi_get import (
    url_get, 
    getSiteProperties, 
    set_browser
) 
from django.core.exceptions import ObjectDoesNotExist
from precios.pi_functions import (
    check_exists, 
    click_element,
    add_page,
    urlSave,
)

def obtener_estructura_enlaces(url_base, palabras_clave_evitar, max_levels, max_time, scheme_que_almacenar):
    df_articulos = pd.DataFrame()
    df_articulos['scheme']                 = ''
    df_articulos['url']                    = ''
    df_articulos['nivel']                  = 0
    # df_articulos['sin_dominio']            = ''
    df_articulos['grupo_1']                = ''
    df_articulos['grupo_2']                = ''
    df_articulos['grupo_3']                = ''
    df_articulos['grupo_4']                = ''
    df_articulos['grupo_5']                = ''
    df_articulos['grupo_6']                = ''
    df_articulos.set_index('url')
    # df_articulos.loc[len(df_articulos)] = {'url': 'laurl'}    

    urls_visitadas = set()  # Almacenar todas las URLs visitadas

    # Configuración de Selenium
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Ejecución sin ventana del navegador
    options.add_argument("--enable-javascript")
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(10)

        
    
    def buscar_enlaces(url, nivel=1, dominio_actual=None):
        # if nivel > max_levels:  # Limitar la recursión
        #     return
        ### Salida x tiempo
        if datetime.now() > max_time:
            # print('salida x tiempo')
            return


        if dominio_actual is None:
            dominio_actual = url


        try:
            driver.get(url)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'a')))
            htmlelement= driver.find_element(By.TAG_NAME, 'html')
            cuenta_down = 0
            while cuenta_down < 2:
                htmlelement.send_keys(Keys.PAGE_DOWN)
                cuenta_down = cuenta_down + 1
                

        except Exception as e:
            print(f"Error al obtener las URLs de {url}: {e} nivel={nivel}")


                
        urls = set()
        

        # for enlace in WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'a'))):        
        for enlace in driver.find_elements(By.TAG_NAME, 'a'):
            try:
                url_enlace = enlace.get_attribute('href')
            except StaleElementReferenceException:
                print("en StaleElementReferenceException")

            action = ActionChains(driver)
            action = ActionChains(driver).send_keys(Keys.PAGE_DOWN)

            action.perform()
            # url_enlace.startswith('http') and \
            # url_enlace.startswith(url) and \
            if url_enlace and \
                contiene_palabras_clave(url_enlace, scheme_que_almacenar) and \
                not contiene_palabras_clave(url_enlace, palabras_clave_evitar) and \
                url_enlace not in df_articulos.values:
                
                parsed_uri = urlparse(url_enlace)
                result = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)

                scheme = parsed_uri.scheme
                sin_dominio = url_enlace.replace(url_base,'')

                grupo_pre = 'grupo_'
                arr_grupos = sin_dominio.split('/')
                num_slashes = len(arr_grupos)
                indice_grupo = 0
                grupo_1 = ''
                grupo_2 = ''
                grupo_3 = ''
                grupo_4 = ''
                grupo_5 = ''
                grupo_6 = ''
                for grupo in arr_grupos:
                    indice_grupo = indice_grupo + 1
                    if grupo !='':
                        if indice_grupo == 1:
                            grupo_1                = grupo
                        if indice_grupo == 2:
                            grupo_2                = grupo
                        if indice_grupo == 3:
                            grupo_3                = grupo
                        if indice_grupo == 4:
                            grupo_4                = grupo
                        if indice_grupo == 5:
                            grupo_5                = grupo
                        if indice_grupo == 6:
                            grupo_6                = grupo
                        if indice_grupo > 6:
                            print(f'indice grupo={indice_grupo}')

                            
        
                df_articulos.loc[len(df_articulos)] = {'url': url_enlace, 'scheme': scheme, 'nivel': nivel, 'grupo_1': grupo_1, 'grupo_2': grupo_2, 'grupo_3': grupo_3, 'grupo_4': grupo_4, 'grupo_5': grupo_5, 'grupo_6': grupo_6}    
                urls.add(url_enlace)

        
        # if url not in estructura_enlaces:
        #     estructura_enlaces[(nivel, url)] = urls
        
        urls_visitadas.update(urls)  # Agregar las URLs visitadas al conjunto global
        # print(f' {len(urls_visitadas)} Nivel={nivel}')
        
        try:
            # Procesar las URLs encontradas
            for nueva_url in urls:
                if nueva_url.startswith(url) :
                    buscar_enlaces(nueva_url, nivel + 1, dominio_actual)
                # else:
                #     print(f'no se sigue hacia {nueva_url}')

        except Exception as e:
            print(f"Error al buscar_enlaces {url}: {e} nivel={nivel}")

    def contiene_palabras_clave(url, palabras_clave):
        return any(palabra in url for palabra in palabras_clave)
    
    def hace_click(browser, como, que_busca):
        if check_exists(browser, como, que_busca):
            click_element(browser,como, que_busca)

    buscar_enlaces(url_base)

    driver.quit()  # Cerrar el navegador después de terminar

    return df_articulos

# Ejemplo de uso con un sitio web inicial

# max_urls_visitadas = 251
max_urls_visitadas  = 500
max_levels          = 3



now = datetime.now()
max_time = now + timedelta(minutes = 2)
## 
# url_base                = ""          ## ___ records / 2 min  & max_levels = _  Time= ___
# url_base     Oka           = "https://legout.cl/"          ## ___ records / 2 min  & max_levels = _  Time= ___
# url_base        NO        = "https://distribuidoraonline.cl/"          ## ___ records / 2 min  & max_levels = _  Time= ___
## En sistema pero no habilitados
# url_base   Oka             = "https://santaritaonline.com/"          ## 101 records / 2 min  & max_levels = 3  Time= 2.20
# url_base   User-agent: *Disallow:             = "https://speedmarkt.cl/"          ## 86 records / 2 min  & max_levels = 3  Time= 2.03
# url_base   Otro dominio https://www.lojavazia.com.br/sitemap.xml             = "https://tienda.icbfs.cl/"          ## 7 records / 2 min  & max_levels = 1  Time= 8.8s
# url_base                = "https://www.supermercadodiez.cl/"          ## 0 records / 2 min  & max_levels = 0  Time= 3 s
# url_base    Sin sitemap            = "https://www.laoferta.cl/"          ## 0 records / 2 min  & max_levels = 0  Time= 12 s
# url_base    Error            = "https://www.fullalmacen.cl/"          ## ___ records / 2 min  & max_levels = _  Time= ___
# url_base    HTTP Error 404: Not Found            = "https://www.comercialbinder.cl/"          ## ___ records / 2 min  & max_levels = _  Time= ___
# url_base    Sin sitemap            = "https://speedmarkt.cl/"          ## ___ records / 2 min  & max_levels = _  Time= ___

###
# url_base   Json type product no encontrado             = "https://www.lider.cl/supermercado"          ## 20 records / 2 min  & max_levels = 2  Time= 26 s
# url_base   Sitemap en otro lugar             = "https://www.laofficina.cl/"          ## 114 records / 2 min  & max_levels = 1  Time= 2m 8.3s
# url_base   HTTP Error 404: Not Found             = "https://megalicores.com/"          ## 0 records / 2 min  & max_levels = _  Time= ___
# url_base   Sin sitemap             = "https://chilepisco.com/"          ## 341 records / 2 min  & max_levels = 1  Time= 44 s
# url_base   Oka             = "https://andina.micoca-cola.cl/"          ## 69 records / 2 min  & max_levels = 1  Time= 53.7
# url_base   Oka             = "https://www.centralvinosylicores.cl/"          ## 255 records / 3 min  & max_levels = 2  Time= 3m 2s
# url_base                = "https://www.espaciovino.com.ar/"          ## 1423 records / 3 min  & max_levels = 2  Time= 3.21
############################
# url_base     Oka           = "https://thewildfoods.com/"          ## 99 records / 2 min  & max_levels = 2  Time= 2.34
####### Buenos 

### Con problemas
# url_base                = "https://www.booz.cl/"                ## 162 records / 2 min  & max_levels = 4  Time= 2.08
# url_base                = "https://cornershopapp.com/es-cl/"    ## 31 records  / 2 min  & max_levels = 2  Time= 1.19
# url_base                = "https://www.knasta.cl/"                  ## 18 records  / 2 min  & max_levels = 3   ####### No hay sitemap

#### Sitemap contiene todos los productos
# url_base                = "https://www.liquidos.cl/"            ## 5 records   / 2 min  & max_levels = 1


# palabras_clave_evitar   = ['/busca', '#','?']

url_base                = "https://www.devop.cl/"          ## ___ records / 2 min  & max_levels = _  Time= ___
# url_base                = "http://127.0.0.1:8000/"          ## ___ records / 2 min  & max_levels = _  Time= ___

palabras_clave_evitar   = ['#','?', '/cart']
protocolos_que_almacenar= ['http', 'mailto']


# adv.crawl(url_base, 'my_output_file.jl', follow_links=True, custom_settings={'CLOSESPIDER_PAGECOUNT': 10})
# enczp = pd.read_json('my_output_file.jl', lines=True)
# enczp.head(5)

url_robots = get_url_robots(url_base)
if not url_robots.empty:
    url_robots.head(10)

    column          = 'directive'
    text_to_search  = 'Sitemap'
    sitemap = get_content_from_column(url_robots, column, text_to_search)
    if not sitemap.empty:
        sitemap = url_robots.loc[url_robots[column] == text_to_search].values[0]
        sitemap_url = sitemap[1]
        print(f'hay sitemap  {sitemap_url}')


        sitemap_content = get_sitemap_from_url(sitemap_url)
        if not sitemap_content.empty:
            print(sitemap_content.head(10))
        else:
            print("NO SE PUDO")
    else:
        print('No hay Sitemap')
        url_robots.head(10)
else:
    print('No hay robots.txt')

# url_base_sitemap = adv.sitemap_to_df('https://www.devop.cl/precios/sitemap.xml')
# url_base_sitemap

# estructura_enlaces = obtener_estructura_enlaces(url_base,  palabras_clave_evitar, max_levels, max_time, protocolos_que_almacenar)

# print(tabulate(estructura_enlaces, headers = 'keys', tablefmt = 'psql'))




INFO:root:Getting: https://www.devop.cl/robots.txt


hay sitemap  https://www.devop.cl/precios/sitemap.xml


INFO:root:Getting https://www.devop.cl/sitemap.xml
INFO:root:Getting https://www.devop.cl/precios/sitemaps/brands-3.xml
INFO:root:Getting https://www.devop.cl/precios/sitemaps/brands-2.xml
INFO:root:Getting https://www.devop.cl/precios/sitemaps/articles-1.xml
INFO:root:Getting https://www.devop.cl/precios/sitemaps/articles-3.xml
INFO:root:Getting https://www.devop.cl/precios/sitemaps/brands-1.xml
INFO:root:Getting https://www.devop.cl/precios/sitemaps/articles-4.xml
INFO:root:Getting https://www.devop.cl/precios/sitemaps/articles-2.xml
INFO:root:Getting https://www.devop.cl/precios/sitemaps/articles-5.xml
INFO:root:Getting https://www.devop.cl/precios/sitemaps/articles-6.xml
INFO:root:Getting https://www.devop.cl/precios/sitemaps/articles-8.xml
INFO:root:Getting https://www.devop.cl/precios/sitemaps/articles-9.xml
INFO:root:Getting https://www.devop.cl/precios/sitemaps/articles-7.xml
INFO:root:Getting https://www.devop.cl/precios/sitemaps/articles-10.xml
INFO:root:Getting https://www.d

                                                 loc changefreq  priority   
0                https://www.devop.cl/precios/estado      daily       0.5  \
1             https://www.devop.cl/precios/cobertura      daily       0.5   
2  https://www.devop.cl/precios/antiguedad_registros      daily       0.5   
3           https://www.devop.cl/precios/brands_all/      daily       0.5   
4                              https://www.devop.cl/    monthly       0.5   
5                         https://www.devop.cl/blog/    monthly       0.5   
6                       https://www.devop.cl/buscar/    monthly       0.5   
7                       https://www.devop.cl/acerca/    monthly       0.5   
8                       https://www.devop.cl/cuenta/    monthly       0.5   
9                        https://www.devop.cl/robot/    monthly       0.5   

                    lastmod                           sitemap   
0                       NaT  https://www.devop.cl/sitemap.xml  \
1                     